In [ ]:
import random
import numpy as np
import random,os
import torch
import pandas as pd
from pathlib import Path
import librosa
from matplotlib import pyplot as plt
import IPython

from PIL import Image
import ipywidgets as widgets
from ipywidgets import Layout
from matplotlib import pyplot as plt

from glob import glob
from IPython.display import display

#Deep learning from pytorch
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import requests

In [ ]:
nan

In [ ]:
import json
json_open = open('/kaggle/input/birdclef-2022/scored_birds.json', 'r')
bird_target = json.load(json_open)
bird_target

train = pd.read_csv("../input/fulllabeldataset/train.csv",index_col=0)
#train = train[train.num <= 3].reset_index(drop=True)
train["filepath"] = "../input/fulllabeldataset/image_128/" + train["path"]
train["sort_index"] = train.filename_id.apply(lambda x: int(x.split("_")[-1])//5)
train["file_id"] = train.filename_id.apply(lambda x: x.split("_")[0])
train['secondary_labels_array'] =  train.secondary_labels.apply(eval)
train["sleng"] = train.secondary_labels_array.apply(lambda x: len(x))
unique_key = sorted(bird_target)
LABEL_IDS = {label: label_id for label_id, label in enumerate(unique_key)}
INV_LABEL_IDS = {val: key for key,val in LABEL_IDS.items()}
print(LABEL_IDS)
train["label_id"] = -1
train.loc[train.primary_label.isin(bird_target),"label_id"] = train.loc[train.primary_label.isin(bird_target),"primary_label"].map(LABEL_IDS)
df_nolabel = train[train.sleng == 0]
df_nolabel['label_array'] =  df_nolabel.label_id.apply(lambda x: np.array([x]))
df_nolabel['num'] = 0
df_nolabel["sec1"] = -1
df_nolabel["sec2"] = -1
df_nolabel["sec3"] = -1

df_label = train[train.sleng > 0]
df_label['labels_id'] = df_label.secondary_labels_array.apply(lambda x: np.vectorize(lambda s: LABEL_IDS[s] if s in unique_key else -1)(x))
df_label.labels_id = df_label.labels_id.apply(lambda x: x[x != -1])
df_label['num'] = df_label.labels_id.apply(lambda x: len(x))
df_label["sec1"] = df_label.labels_id.apply(lambda x: x[0] if len(x) > 0 else -1)
df_label["sec2"] = df_label.labels_id.apply(lambda x: x[1] if len(x) > 1 else -1)
df_label["sec3"] = df_label.labels_id.apply(lambda x: x[2] if len(x) > 2 else -1)

train = pd.concat([df_label,df_nolabel]).sort_values(["file_id","sort_index"]).reset_index(drop=True)

train.loc[train.label_id == train.sec1,"sec1"] = -1
train.loc[train.label_id == train.sec2,"sec2"] = -1
train.loc[train.label_id == train.sec3,"sec3"] = -1

#train["label"] = train[["sec1","sec2","sec3"]].apply(lambda x: np.array([x.sec1,x.sec2,x.sec3],dtype=np.int32),axis=1)
#train["label"] = train.label.apply(lambda x: x[x != -1])
#train["num"] = train.labels_id.apply(lambda x: len(x))


train = train[~((train.label_id == -1)&(train.num==0))].reset_index(drop=True)
train.head(5)

In [ ]:
nan

In [ ]:
furudf = pd.read_csv("../input/annotationdataset-furu/results.txt",header=None).rename(columns={0:"filename_id",1:"primary_label_hand"})
furudfnum1 = pd.read_csv("../input/annotationdataset-furu/results_num1_primary.txt",header=None).rename(columns={0:"filename_id",1:"primary_label_hand"})
ishiidf = pd.read_csv("../input/annotationdataset-ishii/results.txt",header=None).rename(columns={0:"filename_id",1:"primary_label_hand"})
tadokorodf = pd.read_csv("../input/annotationdataset-tadokoro/results.txt",header=None).rename(columns={0:"filename_id",1:"primary_label_hand"})
takemidf = pd.read_csv("../input/annotationdataset-takemi/results.txt",header=None).rename(columns={0:"filename_id",1:"primary_label_hand"})
handlabeldf = pd.concat([furudf,furudfnum1,ishiidf,tadokorodf,takemidf]).drop_duplicates("filename_id")
train = pd.merge(train,handlabeldf,on=["filename_id"],how="left").fillna(0)

furudf_sec = pd.read_csv("../input/annotationdataset-furusec/results_sec1.txt",header=None).rename(columns={0:"filename_id",1:"primary_label_sec1"})
ishiidf1_sec = pd.read_csv("../input/annotationdatasetishiisec/results_sec1.txt",header=None).rename(columns={0:"filename_id",1:"primary_label_sec1"})
ishiidf2_sec = pd.read_csv("../input/annotationdatasetishiisec/results_sec2.txt",header=None).rename(columns={0:"filename_id",1:"primary_label_sec1"})
ishiidf3_sec = pd.read_csv("../input/annotationdatasetishiisec/results_sec3_1.txt",header=None).rename(columns={0:"filename_id",1:"primary_label_sec1"})
takemidf_sec = pd.read_csv("../input/annotationdataset-takemisec/results_sec1.txt",header=None).rename(columns={0:"filename_id",1:"primary_label_sec1"})
handlabeldf_sec1 = pd.concat([furudf_sec,ishiidf1_sec,ishiidf2_sec,ishiidf3_sec,takemidf_sec]).drop_duplicates("filename_id")
train = pd.merge(train,handlabeldf_sec1,on=["filename_id"],how="left").fillna(0)

ishiidf1_sec2 = pd.read_csv("../input/annotationdatasetishiisec/results_sec2.txt",header=None).rename(columns={0:"filename_id",1:"primary_label_sec2"})
ishiidf2_sec2 = pd.read_csv("../input/annotationdatasetishiisec/results_sec3_2.txt",header=None).rename(columns={0:"filename_id",1:"primary_label_sec2"})
handlabeldf_sec2 = pd.concat([ishiidf1_sec2,ishiidf2_sec2]).drop_duplicates("filename_id")
train = pd.merge(train,handlabeldf_sec2,on=["filename_id"],how="left").fillna(0)

ishiidf_sec3 = pd.read_csv("../input/annotationdatasetishiisec/results_sec3_3.txt",header=None).rename(columns={0:"filename_id",1:"primary_label_sec3"})
takemidf_sec3 = pd.read_csv("../input/annotationdataset-takemisec2/results_sec3.txt",header=None).rename(columns={0:"filename_id",1:"primary_label_sec3"})
handlabeldf_sec3 = pd.concat([ishiidf_sec3,takemidf_sec3]).drop_duplicates("filename_id")
train = pd.merge(train,handlabeldf_sec3,on=["filename_id"],how="left").fillna(0)

#skylar houfinに関しては、確率低いもののみハンドラベリング済みなので、高いものは全てTrueにする
train.loc[(train.primary_label.isin(["skylar"]))&(train.num==0),"primary_label_hand"] = "skylar"
train.loc[(train.primary_label.isin(["houfin"]))&(train.num==0),"primary_label_hand"] = "houfin"
train.loc[train.primary_label_hand==0,"primary_label_hand"] = "nolabel"
train.loc[(train.primary_label_sec1==0)&(train.sec1 == -1),"primary_label_sec1"] = "nocall"
train.loc[train.primary_label_sec1==0,"primary_label_sec1"] = "nolabel"
train.loc[(train.primary_label_sec2==0)&(train.sec2 == -1),"primary_label_sec2"] = "nocall"
train.loc[train.primary_label_sec2==0,"primary_label_sec2"] = "nolabel"
train.loc[(train.primary_label_sec3==0)&(train.sec3 == -1),"primary_label_sec3"] = "nocall"
train.loc[train.primary_label_sec3==0,"primary_label_sec3"] = "nolabel"

train.to_csv("handlabeltrain.csv")

In [ ]:
train

In [ ]:
nan

In [ ]:
nan